# Imports

In [13]:
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor as ABR
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

# Load data

In [3]:
df = pd.read_csv('data/ready/train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
1,2,20,0,80.0,9600,0,0,0,0,0,...,0,0,0,0,0,5,2007,0,0,181500
2,3,60,0,68.0,11250,0,0,1,0,0,...,0,0,0,0,0,9,2008,0,0,223500
3,4,70,0,60.0,9550,0,0,1,0,0,...,0,0,0,0,0,2,2006,0,1,140000
4,5,60,0,84.0,14260,0,0,1,0,0,...,0,0,0,0,0,12,2008,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,0,62.0,7917,0,0,0,0,0,...,0,0,0,0,0,8,2007,0,0,175000
1456,1457,20,0,85.0,13175,0,0,0,0,0,...,0,0,1,0,0,2,2010,0,0,210000
1457,1458,70,0,66.0,9042,0,0,0,0,0,...,0,0,3,1,2500,5,2010,0,0,266500
1458,1459,20,0,68.0,9717,0,0,0,0,0,...,0,0,0,0,0,4,2010,0,0,142125


# Train-val split

In [6]:
df_train, df_val = train_test_split(df, test_size=0.2)
df_train.shape, df_val.shape

((1168, 81), (292, 81))

In [9]:
cols_x = list(df_train.drop(columns=['Id', 'SalePrice']).columns)
cols_y = 'SalePrice'
cols_x[:5]

['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street']

# Logit target

In [7]:
def one_plus_log(x):
    return np.log(1 + x)

def one_plus_log_reverse(x):
    return np.exp(x) - 1

df_train['SalePrice'] = df_train['SalePrice'].apply(one_plus_log)
df_val['SalePrice'] = df_val['SalePrice'].apply(one_plus_log)
df_val.head()

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_93312/3151798132.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['SalePrice'] = df_train['SalePrice'].apply(one_plus_log)
/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_93312/3151798132.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['SalePrice'] = df_val['SalePrice'].apply(one_plus_log)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
702,703,60,0,82.0,12438,0,0,1,0,0,...,0,0,0,0,0,7,2006,1,2,12.799178
1137,1138,50,0,54.0,6342,0,0,0,0,0,...,0,0,0,0,0,5,2010,0,0,11.451061
313,314,20,0,150.0,215245,0,0,3,2,0,...,0,0,0,0,0,6,2009,0,0,12.834684
780,781,20,0,63.0,7875,0,0,0,0,0,...,0,0,0,0,0,6,2007,0,0,12.078245
972,973,120,0,55.0,7892,0,0,0,0,0,...,0,0,0,0,0,4,2010,0,0,11.507923


# Search for optimal model

In [11]:
def rmse(y, y_hat):
    return mean_squared_error(y, y_hat) ** 0.5

In [36]:
rf = ABR(random_state=27, loss='square', learning_rate=0.5, n_estimators=100)
rf.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = rf.predict(df_train[cols_x])
y_val_hat = rf.predict(df_val[cols_x])

print('Train RMSE', rmse(y_train_hat, df_train[cols_y]))
print('Test RMSE', rmse(y_val_hat, df_val[cols_y]))

Train RMSE 0.15197156904931144
Test RMSE 0.19818461989813663


# Submit

In [37]:
df_test = pd.read_csv('data/ready/test.csv')
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,4,80.0,11622,0,-1,0,0,0,...,120,0,-1,1,-1,0,6,2010,0,0
1,1462,20,0,81.0,14267,0,-1,1,0,0,...,0,0,-1,-1,2,12500,6,2010,0,0
2,1463,60,0,74.0,13830,0,-1,1,0,0,...,0,0,-1,1,-1,0,3,2010,0,0
3,1464,60,0,78.0,9978,0,-1,1,0,0,...,0,0,-1,-1,-1,0,6,2010,0,0
4,1465,120,0,43.0,5005,0,-1,1,3,0,...,144,0,-1,-1,-1,0,1,2010,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,1,21.0,1936,0,-1,0,0,0,...,0,0,-1,-1,-1,0,6,2006,0,0
1455,2916,160,1,21.0,1894,0,-1,0,0,0,...,0,0,-1,-1,-1,0,4,2006,0,1
1456,2917,20,0,160.0,20000,0,-1,0,0,0,...,0,0,-1,-1,-1,0,9,2006,0,1
1457,2918,85,0,62.0,10441,0,-1,0,0,0,...,0,0,-1,1,1,700,7,2006,0,0


In [38]:
df_test['SalePrice'] = rf.predict(df_test[cols_x])

In [39]:
df_submission = df_test[['Id', 'SalePrice']]
df_submission['SalePrice'] = df_submission['SalePrice'].apply(one_plus_log_reverse)

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_93312/4042444516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submission['SalePrice'] = df_submission['SalePrice'].apply(one_plus_log_reverse)


In [40]:
df_submission

,Id,SalePrice
0,1461,107088.842619
1,1462,130165.706924
2,1463,167201.649733
3,1464,174620.408386
4,1465,200858.875646
...,...,...
1454,2915,86604.278677
1455,2916,86601.569565
1456,2917,157969.905656
1457,2918,111527.777422


In [41]:
df_submission.to_csv('data/results/adaboost_submission.csv', index=False)